In [ ]:
%cd /content/drive/MyDrive/yolo_practice/yolov7_vs_yolov5

# 必要なリポジトリの準備

In [ ]:
!git clone https://github.com/wang-xinyu/tensorrtx.git

In [ ]:
# yolov5 v6.2をクローンしていない場合はクローンする。
!git clone https://github.com/ultralytics/yolov5 -b v6.2

In [ ]:
# yolov7 latestをクローンしていない場合はクローンする。
!git clone https://github.com/WongKinYiu/yolov7

# .wtsファイルへの変換

google colabで行う。

## yolov5s, yolov5n

gen_wts.pyをyolov5下にコピーする。

In [ ]:
!cp tensorrtx/yolov5/gen_wts.py yolov5

In [ ]:
%cd yolov5

In [ ]:
!mkdir weights

yolov5/weights下に学習済みモデルをコピーする。

In [ ]:
!ls weights

yolov5n_best.pt  yolov5s_best.pt


以下のオプションを設定し、.ptファイルを.wtsファイルに変換する。  
- w: 学習済みモデル(.ptファイル)のパス
- o: 変換後の.wtsファイルのパス

In [ ]:
!python3 gen_wts.py -w weights/yolov5s_best.pt -o weights/yolov5s_best.wts

In [ ]:
!python3 gen_wts.py -w weights/yolov5n_best.pt -o weights/yolov5n_best.wts

## yolov7

In [ ]:
%cd /content/drive/MyDrive/yolo_practice/yolov7_vs_yolov5

/content/drive/MyDrive/yolo_practice/yolov7_vs_yolov5


gen_wts.pyをyolov7下にコピーする。

In [ ]:
!cp tensorrtx/yolov7/gen_wts.py yolov7

In [ ]:
%cd yolov7

/content/drive/MyDrive/yolo_practice/yolov7_vs_yolov5/yolov7


In [ ]:
!mkdir weights

yolov5/weights下に学習済みモデルをコピーする。

In [ ]:
!ls weights

yolov7_tiny_best.pt


以下のオプションを設定し、.ptファイルを.wtsファイルに変換する。  
- w: 学習済みモデル(.ptファイル)のパス
- o: 変換後の.wtsファイルのパス

In [ ]:
!python3 gen_wts.py -w weights/yolov7_tiny_best.pt -o weights/yolov7_tiny_best.wts

# .engineファイルへの変換

Jetsonで行う。  

## yolov5s, yolov5n

1. tensorrtx/yolov5をjetsonにコピーする。



2. yololayer.h の20-22行目を適宜変更する。今回はCLASS_NUM = 1, INPUT_H = 416, INPUT_W = 416となるように修正する。


```yololayer.h
#ifndef _YOLO_LAYER_H
#define _YOLO_LAYER_H

#include <vector>
#include <string>
#include <NvInfer.h>
#include "macros.h"

namespace Yolo
{
    static constexpr int CHECK_COUNT = 3;
    static constexpr float IGNORE_THRESH = 0.1f;
    struct YoloKernel
    {
        int width;
        int height;
        float anchors[CHECK_COUNT * 2];
    };
    static constexpr int MAX_OUTPUT_BBOX_COUNT = 1000;
    static constexpr int CLASS_NUM = 1;
    static constexpr int INPUT_H = 416;  // yolov5's input height and width must be divisible by 32.
    static constexpr int INPUT_W = 416;
```

3. tensorrtx/yolov5に移動してターミナルを開き、以下のコマンドを実行する。
```
mkdir build
```
4. buildディレクトリに.wtsファイルをコピーする。
5. .wtsファイルの名前をyolov5s.wts(yolov5nの場合はyolov5n.wts)に変更する。
6. 以下のコマンドを実行する。
```
cd build
cmake ..
make
```
7. yolov5sの場合は以下のコマンドを実行する。
```
sudo ./yolov5 -s yolov5s.wts yolov5s.engine s
```
8. yolov5nの場合は以下のコマンドを実行する。
```
sudo ./yolov5 -s yolov5n.wts yolov5s.engine n
```



## yolov7

1. tensorrtx/yolov5をjetsonにコピーする。
2. tensorrtx/yolov7/include/config.hのkNumClass, kInputH, kInputW, kConfThreshを変更する。今回はkNumClass = 1, kInputH = 416, kInputW = 416, kConfThresh = 0.25となるように修正する。

```
#pragma once

/* --------------------------------------------------------
 * These configs are related to tensorrt model, if these are changed,
 * please re-compile and re-serialize the tensorrt model.
 * --------------------------------------------------------*/

// For INT8, you need prepare the calibration dataset, please refer to
// https://github.com/wang-xinyu/tensorrtx/tree/master/yolov7#int8-quantization
#define USE_FP16  // set USE_INT8 or USE_FP16 or USE_FP32

// These are used to define input/output tensor names,
// you can set them to whatever you want.
const static char* kInputTensorName = "data";
const static char* kOutputTensorName = "prob";

const static int kNumClass = 1;
const static int kBatchSize = 1;

// Yolo's input width and height must by divisible by 32
const static int kInputH = 416;
const static int kInputW = 416;

// Maximum number of output bounding boxes from yololayer plugin.
// That is maximum number of output bounding boxes before NMS.
const static int kMaxNumOutputBbox = 1000;

const static int kNumAnchor = 3;

// The bboxes whose confidence is lower than kIgnoreThresh will be ignored in yololayer plugin.
const static float kIgnoreThresh = 0.1f;

/* --------------------------------------------------------
 * These configs are not related to tensorrt model, if these are changed,
 * please re-compile, but no need to re-serialize the tensorrt model.
 * --------------------------------------------------------*/

// NMS overlapping thresh and final detection confidence thresh
const static float kNmsThresh = 0.45f;
const static float kConfThresh = 0.25f;

const static int kGpuId = 0;

// If your image size is larger than 4096 * 3112, please increase this value
const static int kMaxInputImageSize = 4096 * 3112;
```

3. tensorrtx/yolov7に移動してターミナルを開き、以下のコマンドを実行する。
```
mkdir build
```
4. buildディレクトリに.wtsファイルをコピーする。
5. .wtsファイルの名前をyolov7_tiny.wtsに変更する。
6. 以下のコマンドを実行する。
```
cd build
cmake ..
make
```
7. 以下のコマンドを実行する。
```
sudo ./yolov7 -s yolov7_tiny.wts yolov7_tiny.engine t
```